### Preprocessing and Cleaning for hud.csv

#### Import Libraries

In [2]:
import warnings
import numpy as np
import pandas as pd
import plotly.express as px


warnings.filterwarnings("ignore")

#### Import Data

In [ ]:
us_cities = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/us-cities-top-1k.csv")

In [ ]:
us_cities.head()

In [ ]:
hud_df = pd.read_csv('../data/hud/hud_yearly.csv')

In [ ]:
hud_df

In [ ]:
hud_df.sample(5, random_state=24)

#### Clean "us_cities" data
Replace states with abbreviations

In [ ]:
states = {
    'Alaska': 'AK',
    'Alabama': 'AL',
    'Arkansas': 'AR',
    'Arizona': 'AZ',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'District of Columbia': 'DC',
    'Delaware': 'DE',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Iowa': 'IA',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Massachusetts': 'MA',
    'Maryland':'MD',
    'Maine': 'ME',
    'Michigan': 'MI',
    'Minnesota':'MN',
    'Missouri': 'MO',
    'Mississippi': 'MS',
    'Montana': 'MT',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Nebraska': 'NE',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'Nevada': 'NV',
    'New York': 'NY',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon':'OR',
    'Pennsylvania': 'PA',
    'Rhode Island': 'RI',
    'South Carolina':'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas':'TX',
    'Utah': 'UT',
    'Virginia': 'VA',
    'Vermont': 'VT',
    'Washington': 'WA',
    'Wisconsin': 'WI',
    'West Virginia': 'WV',
    'Wyoming': 'WY'
}

In [ ]:
us_cities["State"] = us_cities["State"].map(states)

In [ ]:
us_cities.sort_values(by=["State", "City"], inplace=True)

In [ ]:
us_cities.reset_index(drop=True, inplace=True)

In [ ]:
us_cities = us_cities[["City", "State", "lat", "lon"]]

In [ ]:
us_cities.head()

In [ ]:
us_cities.sample(5, random_state=24)

#### Clean "hud_df"
Reshape data from wide to long format

In [ ]:
hud_df.rename(columns={'Change': '1Y', 
                       '2Y Change': '2Y', 
                       '3Y Change': '3Y', 
                       '4Y Change': '4Y'}, 
              inplace=True)

In [ ]:
# reshape data
hud_df = pd.melt(hud_df,
                 id_vars=['City', 'State', 'Bedrooms', 'Year', 'Avg Rent',
                          'YoY', 'Yo2Y', 'Yo3Y', 'Yo4Y'],
                 var_name='Yearly Difference', 
                 value_vars=['1Y', '2Y', '3Y', '4Y'],
                 value_name='Price Change')

hud_df = pd.melt(hud_df,
                 id_vars=['City', 'State', 'Bedrooms', 'Year', 'Avg Rent',
                          'Yearly Difference', 'Price Change'],
                 var_name='Growth',
                 value_vars=['YoY', 'Yo2Y', 'Yo3Y', 'Yo4Y'],
                 value_name='Percent Change')

In [ ]:
hud_df.sample(5, random_state=24)

###### Drop missing and duplicate values
1. Drop missing values
2. Identify duplicate values and remove them from dataset

In [ ]:
# Drop missing and duplicate values
hud_df.dropna(inplace=True)
hud_df.drop_duplicates(inplace=True)

Create new column **is_duplicate** to flag duplicate values in **Yearly Difference** and **Growth** columns. 
Duplicates values are those where values for **Yearly Difference** and **Growth** don't match, i.e. **4Y** and **Yo2Y**.

In [ ]:
# Create "is_duplicate" column flagging duplicate values

conditions = [
    (hud_df['Yearly Difference'] == '1Y') & (hud_df['Growth'] != 'YoY'), 
    (hud_df['Yearly Difference'] == '2Y') & (hud_df['Growth'] != 'Yo2Y'),
    (hud_df['Yearly Difference'] == '3Y') & (hud_df['Growth'] != 'Yo3Y'), 
    (hud_df['Yearly Difference'] == '4Y') & (hud_df['Growth'] != 'Yo4Y')
    ]
              
values = ['Yes', 
          'Yes', 
          'Yes', 
          'Yes']

hud_df['is_duplicate'] = np.select(conditions, values, default="No")

In [ ]:
hud_df.sample(5, random_state=24)

In [ ]:
# Remove duplicate values from dataset
hud_df = hud_df[hud_df['is_duplicate'] == 'No']

In [ ]:
hud_df.reset_index(drop=True, inplace=True)

In [ ]:
hud_df.sample(5, random_state=24)

###### Preprocess dataset by:
1. Creating **Time Range** column
2. Converting numbers in **Price Change** from float to int
3. Removing columns that are unnecessary for analysis

In [ ]:
# Identify time ranges and create "Time Range" column

conditions = [
    (hud_df['Year'] == 2019) & (hud_df['Yearly Difference'] == '1Y') & (hud_df['Growth'] == 'YoY'), 
    (hud_df['Year'] == 2020) & (hud_df['Yearly Difference'] == '1Y') & (hud_df['Growth'] == 'YoY'),
    (hud_df['Year'] == 2020) & (hud_df['Yearly Difference'] == '2Y') & (hud_df['Growth'] == 'Yo2Y'), 
    (hud_df['Year'] == 2021) & (hud_df['Yearly Difference'] == '1Y') & (hud_df['Growth'] == 'YoY'),
    (hud_df['Year'] == 2021) & (hud_df['Yearly Difference'] == '2Y') & (hud_df['Growth'] == 'Yo2Y'),
    (hud_df['Year'] == 2021) & (hud_df['Yearly Difference'] == '3Y') & (hud_df['Growth'] == 'Yo3Y'),
    (hud_df['Year'] == 2022) & (hud_df['Yearly Difference'] == '1Y') & (hud_df['Growth'] == 'YoY'),
    (hud_df['Year'] == 2022) & (hud_df['Yearly Difference'] == '2Y') & (hud_df['Growth'] == 'Yo2Y'),
    (hud_df['Year'] == 2022) & (hud_df['Yearly Difference'] == '3Y') & (hud_df['Growth'] == 'Yo3Y'),
    (hud_df['Year'] == 2022) & (hud_df['Yearly Difference'] == '4Y') & (hud_df['Growth'] == 'Yo4Y')
    ]
              
values = ['2018 to 2019', 
          '2019 to 2020', 
          '2018 to 2020', 
          '2020 to 2021',
          '2019 to 2021',
          '2018 to 2021',
          '2021 to 2022',
          '2020 to 2022',
          '2019 to 2022',
          '2018 to 2022']

hud_df['Time Range'] = np.select(conditions, values, default="Unknown")

In [ ]:
hud_df.sample(5, random_state=24)

In [ ]:
# Convert numbers from float to int
hud_df['Price Change'] = hud_df['Price Change'].astype('int')

In [ ]:
# Remove unnecessary columns

hud_df = hud_df[['City', 'State', 'Bedrooms', 'Year', 'Time Range', 
                 'Growth', 'Avg Rent', 'Price Change', 'Percent Change']]

In [ ]:
hud_df.head()

#### Combine "hud_df" and "us_cities"

In [ ]:
hud_df = pd.merge(hud_df, us_cities, on=["City", "State"], how="left")

In [ ]:
hud_df = hud_df[["City", "State", "lat", "lon",
                 "Bedrooms", "Year", "Time Range", "Growth",
                 "Avg Rent", "Price Change", "Percent Change"]]

In [ ]:
hud_df.head()

In [ ]:
hud_df.to_csv('../data/hud/hud_final.csv', index=False)

### Hud_yearly Cleaning

In [13]:
hud_df = pd.read_csv('../data/hud/hud_final.csv', parse_dates=["Year"])

In [14]:
hud_df.head()

,City,State,lat,lon,Bedrooms,Year,Time Range,Growth,Avg Rent,Price Change,Percent Change
0,Phoenix,AZ,33.448377,-112.074037,studio,2019-01-01,2018 to 2019,YoY,847,103,13.84
1,Los Angeles,CA,34.052234,-118.243685,studio,2019-01-01,2018 to 2019,YoY,1279,121,10.45
2,Sacramento,CA,38.581572,-121.494400,studio,2019-01-01,2018 to 2019,YoY,952,99,11.61
3,San Francisco,CA,37.774929,-122.419415,studio,2019-01-01,2018 to 2019,YoY,2197,375,20.58
4,Chicago,IL,41.878114,-87.629798,studio,2019-01-01,2018 to 2019,YoY,956,41,4.48


In [15]:
onebd = hud_df.query("Bedrooms=='1' and Growth=='YoY'" )
onebd.sample(5, random_state=24)

,City,State,lat,lon,Bedrooms,Year,Time Range,Growth,Avg Rent,Price Change,Percent Change
71,Los Angeles,CA,34.052234,-118.243685,1,2022-01-01,2021 to 2022,YoY,1747,143,8.92
65,Boston,MA,42.360082,-71.058880,1,2021-01-01,2020 to 2021,YoY,1986,62,3.22
79,Seattle,WA,47.606209,-122.332071,1,2022-01-01,2021 to 2022,YoY,1881,142,8.17
42,Sacramento,CA,38.581572,-121.494400,1,2019-01-01,2018 to 2019,YoY,1072,104,10.74
45,Boston,MA,42.360082,-71.058880,1,2019-01-01,2018 to 2019,YoY,1900,339,21.72


In [16]:
onebd = hud_df.query("Bedrooms=='1' and Growth=='YoY'" )
onebd["Year"] = onebd["Year"].dt.strftime("%Y")

In [17]:
onebd.sample(5, random_state=24)

,City,State,lat,lon,Bedrooms,Year,Time Range,Growth,Avg Rent,Price Change,Percent Change
71,Los Angeles,CA,34.052234,-118.243685,1,2022,2021 to 2022,YoY,1747,143,8.92
65,Boston,MA,42.360082,-71.058880,1,2021,2020 to 2021,YoY,1986,62,3.22
79,Seattle,WA,47.606209,-122.332071,1,2022,2021 to 2022,YoY,1881,142,8.17
42,Sacramento,CA,38.581572,-121.494400,1,2019,2018 to 2019,YoY,1072,104,10.74
45,Boston,MA,42.360082,-71.058880,1,2019,2018 to 2019,YoY,1900,339,21.72


In [39]:
cities = sorted(onebd['City'].unique().tolist()) 

fig = px.line(onebd, 
              x="Year",
              y="Percent Change", 
              color="City", 
              custom_data=["Price Change"], 
              category_orders={"City":cities},
              labels={"Percent Change":"YoY"},
              template= 'ygridoff'
              )

fig.update_traces(
    mode="markers+lines", 
    hovertemplate=None
)

fig.update_xaxes(dtick=1)

fig.update_layout(
    title=dict(
        text="Year-over-year change in median monthly one bedroom apartment rent",
        x=0.5,
        y=0.95,
        xanchor="center",
        yanchor="top"
    ),
    yaxis_ticksuffix="%",
    hovermode='x unified')

fig.show()

In [55]:
cities = sorted(onebd['City'].unique().tolist()) 

fig = px.line(onebd, 
              x="Year",
              y="Avg Rent", 
              color="City", 
              custom_data=["Price Change"], 
              category_orders={"City":cities},
              labels={"Avg Rent":"Median Monthly Rent"},
              template= 'ygridoff'
              )

fig.update_traces(
    mode="markers+lines", 
    hovertemplate=None
)

fig.update_xaxes(dtick=1)

fig.update_layout(
    title=dict(
        text="Median monthly rent for one bedroom apartments",
        x=0.5,
        y=0.95,
        xanchor="center",
        yanchor="top"
    ),
    yaxis_tickformat="$",
    hovermode='x unified')

fig.show()

In [56]:
onebd.head()

,City,State,lat,lon,Bedrooms,Year,Time Range,Growth,Avg Rent,Price Change,Percent Change
40,Phoenix,AZ,33.448377,-112.074037,1,2019,2018 to 2019,YoY,958,90,10.37
41,Los Angeles,CA,34.052234,-118.243685,1,2019,2018 to 2019,YoY,1517,133,9.61
42,Sacramento,CA,38.581572,-121.494400,1,2019,2018 to 2019,YoY,1072,104,10.74
43,San Francisco,CA,37.774929,-122.419415,1,2019,2018 to 2019,YoY,2720,465,20.62
44,Chicago,IL,41.878114,-87.629798,1,2019,2018 to 2019,YoY,1076,32,3.07


In [57]:
onebd22 = onebd.query("Bedrooms=='1' and Year=='2022'")

In [58]:
onebd22.head()

,City,State,lat,lon,Bedrooms,Year,Time Range,Growth,Avg Rent,Price Change,Percent Change
70,Phoenix,AZ,33.448377,-112.074037,1,2022,2021 to 2022,YoY,1467,376,34.46
71,Los Angeles,CA,34.052234,-118.243685,1,2022,2021 to 2022,YoY,1747,143,8.92
72,Sacramento,CA,38.581572,-121.494400,1,2022,2021 to 2022,YoY,1400,172,14.01
73,San Francisco,CA,37.774929,-122.419415,1,2022,2021 to 2022,YoY,2665,34,1.29
74,Chicago,IL,41.878114,-87.629798,1,2022,2021 to 2022,YoY,1255,94,8.10


In [59]:
# Sort cities in alphabetical order
hud22_df = hud_df.query("Year==2022 and Growth=='YoY'" )
cities = sorted(hud22_df['City'].unique().tolist()) 

fig = px.bar(hud22_df, 
             x="Percent Change", 
             y="City", 
             custom_data=["Avg Rent", "Price Change", "Year"],
             barmode="group",
             color="Bedrooms", 
             category_orders={"City": cities, 
                              "Bedrooms": ["studio", "1", "2", "3", "4"]},
             labels={'Percent Change': 'YoY'},
             height=700,
             orientation='h',
             template= 'ygridoff')

fig.update_traces(hovertemplate=
                  "<b>%{y}</b><br><br>" +
                  "Year: %{customdata[2]}<br>" +
                  "Median Rent: %{customdata[0]:$,.0f}<br>" +
                  "Difference: %{customdata[1]:$,.0f}<br>" +
                  "YoY: %{x}<br>" +
                  "<extra></extra>",
                  )

fig.update_layout(
    title=dict(
        text="Year-over-year change in median monthy apartment rent from 2021 to 2022 by apartment size",
        x=0.5,
        y=0.95,
        xanchor="center",
        yanchor="top"
    ),
    hoverlabel=dict(
        font_size=16,
        font_family="Rockwell"
    ),
    xaxis_ticksuffix="%"
    
)

fig.show()